# Precomputing values for use in fits of Stan models

Because of the way Stan works, it is necessary to compute some values in advance which can then be passed into the fit an interpolated over. The precomputed values will be different for different sets of source distances, and therefore different catalogues. 

Here we show how to compute the values for the SBG catalogue, but it is exactly the same for all cases, just changing the input label.

For ease, all the precomputed table files used are provided for use in this repository.

In [1]:
import h5py

from fancy import Data, Model, Analysis
from fancy.detector.auger2014 import detector_properties

/Users/fran/projects/bayes/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/fran/projects/bayes/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
# Define file containing catalogue information
source_file = '../data/sourcedata.h5'

# Path to Stan files
stan_path = '../stan/'

# File in which to store precomputation
table_file = 'output/precomputation_storage.h5'

In [5]:
# What sources do we have info on?
with h5py.File(source_file, 'r') as f:
    for key in f:
        print(key)

2FHL_250Mpc
SBG_23
swift_BAT_213


## Data, model and analysis

The precomputed values depend on the source locations and the detector parameters. We also need to define a model in order to pass $E_\rm{th}$ into the energy interpolation tables.

The Analysis object brings together data and model inputs and provides an interface to do the precomputation.

In [6]:
data = Data()
data.add_source(source_file, 'SBG_23') 
data.source.select_sources([0, 1]) # just select 2 sources to speed up demo
data.add_detector(detector_properties)

model_name = 'joint_model.stan'
model = Model(model_filename = model_name, include_paths = stan_path)
model.input(Eth = 52) # EeV


precomp_output = 'output/testing_precomputation.h5'
summary = b'Demonstration of precomputation.' 
analysis = Analysis(data, model, analysis_type = 'joint', 
                    filename = precomp_output, summary = summary)

## Exposure integral precomputation
See Equation A6 in Capel & Mortlock (2019). Interpolated over to calculate $\bar{N}$ in the fit when $\kappa$ is unknown a priori.

$$
\epsilon_k = \int \rm{d} \omega \ p(\omega | \varpi_k, \kappa) \epsilon(\omega)
$$



In [7]:
analysis.build_tables(fit_only = True)
analysis.tables.save(table_file)

## Energy interpolation
Used to speed up fits, can solve the continuous energy loss DE, but can also interpolate over precomputed values to get $E$ given $\tilde{E}$ for a given $D$ and vice versa.

Speed up for a typical fit is from ~hours to ~minutes. I spent a considerable amount of time verifying the results are consistentent between the two methods. 

In [8]:
analysis.build_energy_table(table_file = table_file)